### CS513 Data Cleansining AirBnB dataset

Step 1: Open Refine Processing, base dataset: Source: https://uofi.app.box.com/s/y7123ku1dpzd4m3126sqvpa2agvv08hv

Step 2: Python Analysis & Cleansing notebook: **02. data-cleansing.ipynb**

Step 3: Drop Tables that exist **03_drop_all_tables.sql**, while the import process due to the primary keys, foreign keys and uniqueness constraints will ensure data quality, this script will ensure all data imported will be correctly handled

Step 4: Create Base Table leveraging sql script: **04_baseTable_Create.sql**

Step 5: Populate Base Table **05_PopulateBaseTable.ipynb**

At this point in the data cleansing process, the original dataset has gone through:

01. OpenRefine - initial data cleansing

02. data-cleansing notebook 

03. 03_TableGenerationAndPopulation

The next step in the process is ingesting the dataset into SQL Server.  SQL Server Seed scripts are limited to 1000 records in a seed script.  To get around this requirement, and to ensure the data is inserted intelligently, Step 03b drops tables if they exist.  This step is followed by insertion of the data into a base empty table with no uniqueness constraints, primary or foreign keys.

The data is next loaded into addtional table host, neighbourhood, location, room_type, listing, and a review table.  These tables have primary & foreign key constraints, as well as uniqueness constraints.

03.a drop_all_tables.sql

03.b 03b_baseTable_Create.sql - this notebook runs this script

03.c Populate base table - this notebook runs this script

03.d Populate host table - this notebook runs this script

03.e Populate neighbourhood table - this notebook runs this script

03.f Populate location table - this notebook runs this script

03.g Populate room type

03.h Populate listing table

03.i Populate review table

03.j Query Tables for dataset - this step is to generate a csv file from the dataset, however the sql queries against the data will be handled via sql server.

In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("02_airbnb_step2_complete.csv")
df.columns
print('reviews_per_month: ' + str(df['reviews_per_month'].isna().sum()))
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
print('reviews_per_month: ' + str(df['reviews_per_month'].isna().sum()))
print('--')
print('number_of_reviews: ' + str(df['number_of_reviews'].isna().sum()))
df['number_of_reviews'] = df['number_of_reviews'].fillna(0)
print('number_of_reviews: ' + str(df['number_of_reviews'].isna().sum()))
print('--')
print('host_name_secondary: ' + str(df['host_name_secondary'].isna().sum()))
df['host_name_secondary'] = df['host_name_secondary'].fillna('')
print('host_name_secondary: ' + str(df['host_name_secondary'].isna().sum()))



reviews_per_month: 0
reviews_per_month: 0
--
number_of_reviews: 0
number_of_reviews: 0
--
host_name_secondary: 7052
host_name_secondary: 0


In [3]:
df.loc[df['latitude'] == 41.96084084]


id                                          name  host_id  \
55  265238  Bohemian Chic In The Heart Of Lincoln Square  1382472   

   host_name_primary host_name_secondary neighbourhood   latitude  longitude  \
55               Ani                      North Center  41.960841  -87.68463   

          room_type  price  ...  last_review  reviews_per_month  \
55  Entire home/apt     75  ...          NaN                0.0   

   calculated_host_listings_count  availability_365  \
55                              1               211   

    quality_minimum_nights_long  quality_availability_365  \
55                        False                     False   

    quality_price_outlier  quality_last_review_ismissing  \
55                  False                           True   

    quality_reviews_per_month_ismissing  quality_num_missing  
55                                 True                    2  

[1 rows x 22 columns]

In [4]:
df.dtypes

id                                       int64
name                                    object
host_id                                  int64
host_name_primary                       object
host_name_secondary                     object
neighbourhood                           object
latitude                               float64
longitude                              float64
room_type                               object
price                                    int64
minimum_nights                           int64
number_of_reviews                        int64
last_review                             object
reviews_per_month                      float64
calculated_host_listings_count           int64
availability_365                         int64
quality_minimum_nights_long               bool
quality_availability_365                  bool
quality_price_outlier                     bool
quality_last_review_ismissing             bool
quality_reviews_per_month_ismissing       bool
quality_num_m

In [5]:
server = 'mmm-adf-optimization.database.windows.net, 1433' 
database = 'airbnb' 
username = 'adfsa' 
password = 'R0ck&R0ll' 

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

## 03a drop_all_tables.sql
If a table exists, it will be dropped from the sql server database

In [6]:
sql = '''
    IF OBJECT_ID('airbnb_base') IS NOT NULL
        DROP TABLE airbnb_base;

    IF OBJECT_ID('review_summary') IS NOT NULL
        DROP TABLE review_summary;

    IF OBJECT_ID('listing') IS NOT NULL
        DROP TABLE listing;

    IF OBJECT_ID('location') IS NOT NULL
        DROP TABLE location;

    IF OBJECT_ID('neighbourhood') IS NOT NULL
        DROP TABLE neighbourhood;

    IF OBJECT_ID('neighbourhood') IS NOT NULL
        DROP TABLE neighbourhood;

    IF OBJECT_ID('host') IS NOT NULL
        DROP TABLE host;
    
    IF OBJECT_ID('room_type') IS NOT NULL
        DROP TABLE room_type;
'''

In [7]:
cnxn.execute(sql)

cnxn.commit()

## 03.b_baseTable_Create.sql

In [8]:
sql = '''

IF OBJECT_ID('airbnb_base') IS NOT NULL
    DROP TABLE airbnb_base;

CREATE TABLE airbnb_base (
id VARCHAR(255) NULL,
name VARCHAR(255) NULL,
host_id VARCHAR(255) NULL,
host_name_primary VARCHAR(255) NULL,
host_name_secondary VARCHAR(255) NULL,
neighbourhood VARCHAR(255) NULL,
latitude decimal(10,8),
longitude decimal(10,8),
room_type VARCHAR(255) NULL,
price VARCHAR(255) NULL,
minimum_nights VARCHAR(255) NULL,
number_of_reviews int NULL,
last_review date NULL,
reviews_per_month DECIMAL(5,2) NULL,
calculated_host_listings_count int NULL,
availability_365 VARCHAR(255) NULL,
quality_minimum_nights_long VARCHAR(255) NULL,
quality_availability_365 VARCHAR(255) NULL,
quality_price_outlier VARCHAR(255) NULL,
quality_last_review_ismissing VARCHAR(255) NULL,
quality_reviews_per_month_ismissing VARCHAR(255) NULL,
quality_num_missing VARCHAR(255) NULL
);


'''

In [9]:
cnxn.execute(sql)

cnxn.commit()

## 03.c. Populate SQL Base Table

In [10]:

# Insert Dataframe into SQL Server:
try:
    for index, row in df.iterrows():
        if pd.isna(row.last_review ):
            print('null vlaue for last review')
            print(row.id, row['name'], row.host_id, row.host_name_primary, row.host_name_secondary, row.neighbourhood, row.latitude,row.longitude,row.room_type, row.price, row.minimum_nights, row.number_of_reviews, row.last_review, row.reviews_per_month, row.calculated_host_listings_count, row.availability_365, row.quality_minimum_nights_long, row.quality_availability_365, row.quality_price_outlier,row.quality_last_review_ismissing,  row.quality_reviews_per_month_ismissing, row.quality_num_missing)
            cursor.execute("INSERT INTO dbo.airbnb_base (id, name, host_id, host_name_primary, host_name_secondary,neighbourhood, latitude, longitude, room_type, price,minimum_nights, number_of_reviews, last_review,reviews_per_month, calculated_host_listings_count,availability_365, quality_minimum_nights_long,quality_availability_365, quality_price_outlier,quality_last_review_ismissing, quality_reviews_per_month_ismissing,quality_num_missing) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.id, row['name'], row.host_id, row.host_name_primary, row.host_name_secondary, row.neighbourhood, row.latitude,row.longitude,row.room_type, row.price, row.minimum_nights, row.number_of_reviews, None, row.reviews_per_month, row.calculated_host_listings_count, row.availability_365, row.quality_minimum_nights_long, row.quality_availability_365, row.quality_price_outlier,row.quality_last_review_ismissing,  row.quality_reviews_per_month_ismissing, row.quality_num_missing    )
        else:
            print(row.id, row['name'], row.host_id, row.host_name_primary, row.host_name_secondary, row.neighbourhood, row.latitude,row.longitude,row.room_type, row.price, row.minimum_nights, row.number_of_reviews, row.last_review, row.reviews_per_month, row.calculated_host_listings_count, row.availability_365, row.quality_minimum_nights_long, row.quality_availability_365, row.quality_price_outlier,row.quality_last_review_ismissing,  row.quality_reviews_per_month_ismissing, row.quality_num_missing)
            cursor.execute("INSERT INTO dbo.airbnb_base (id, name, host_id, host_name_primary, host_name_secondary,neighbourhood, latitude, longitude, room_type, price,minimum_nights, number_of_reviews, last_review,reviews_per_month, calculated_host_listings_count,availability_365, quality_minimum_nights_long,quality_availability_365, quality_price_outlier,quality_last_review_ismissing, quality_reviews_per_month_ismissing,quality_num_missing) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.id, row['name'], row.host_id, row.host_name_primary, row.host_name_secondary, row.neighbourhood, row.latitude,row.longitude,row.room_type, row.price, row.minimum_nights, row.number_of_reviews, row.last_review, row.reviews_per_month, row.calculated_host_listings_count, row.availability_365, row.quality_minimum_nights_long, row.quality_availability_365, row.quality_price_outlier,row.quality_last_review_ismissing,  row.quality_reviews_per_month_ismissing, row.quality_num_missing    )
except Exception as e:
    print(e)
cnxn.commit()
cursor.close()

2384 Hyde Park Walk To Uchicago/ Theological Seminary 2613 Rebecca  Hyde Park 41.7888649 -87.58670891 Private room 50 2 137 2018-11-12 2.92 1 163 False False False False False 0
6715 Lincoln Park Oasis Unit 2 Only 15365 Reem  O'hare 41.92926222 -87.66009125 Entire home/apt 255 4 93 2018-08-12 0.81 2 303 False False False False False 0
7126 Tiny Studio Apartment 94 Walk Score 17928 Sarah  West Town 41.90289494 -87.68182159999999 Entire home/apt 80 2 321 2018-10-29 2.81 1 362 False False False False False 0
9811 Barbara'S Hideaway Old Town 33004 At Home Inn  Lincoln Park 41.91768924 -87.63787944 Entire home/apt 150 3 33 2018-10-14 0.59 9 350 False False False False False 0
10610 3 Comforts Of Cooperative Living 2140 Lois Ed Hyde Park 41.79708495 -87.59194894 Private room 35 2 31 2018-07-29 0.63 5 179 False False False False False 0
10945 The Biddle House #1 33004 At Home Inn  Lincoln Park 41.91182685 -87.63999816 Entire home/apt 215 3 9 2018-11-05 0.16 9 357 False False False False False

### 03.d. Populate host table

In [11]:
sql = '''

IF OBJECT_ID('host') IS NOT NULL
    DROP TABLE host;

CREATE TABLE host (
    host_id int ,
    host_name_primary varchar(255) NOT NULL,
	host_name_secondary varchar(255)  NULL,
	calculated_host_listings_count int NOT NULL
    PRIMARY KEY (host_id),
);

INSERT INTO host
select distinct host_id, host_name_primary, host_name_secondary, calculated_host_listings_count from [dbo].[airbnb_base]
'''

In [12]:
cnxn.execute(sql)

cnxn.commit()

### 03.e. Populate neighbourhood table

In [13]:
sql = '''
IF OBJECT_ID('neighbourhood') IS NOT NULL
    DROP TABLE neighbourhood;

CREATE TABLE neighbourhood (
    neighbourhood_id int IDENTITY ,
	neighbourhood varchar(255) NOT NULL,
    PRIMARY KEY (neighbourhood_id),
	CONSTRAINT UC_neighbourhood UNIQUE (neighbourhood )
);

INSERT INTO neighbourhood
select distinct [neighbourhood] from [dbo].[airbnb_base]
'''

In [14]:
cnxn.execute(sql)

cnxn.commit()

### 03.f. Populate location table

In [15]:
sql = '''
IF OBJECT_ID('location') IS NOT NULL
    DROP TABLE location;

CREATE TABLE location (
    location_id int IDENTITY(1,1),
    latitude decimal(10,8) NOT NULL,
    longitude decimal(10,8) NOT NULL,
	neighbourhood_id int NOT NULL
    PRIMARY KEY (location_id),
	FOREIGN KEY (neighbourhood_id)	REFERENCES neighbourhood(neighbourhood_id),
);

--note the same location id is reused ---
INSERT INTO location
select distinct 
[latitude], [longitude], [neighbourhood].neighbourhood_id
from [dbo].[airbnb_base]
LEFT JOIN [dbo].[neighbourhood] ON [airbnb_base].neighbourhood = [neighbourhood].neighbourhood



'''

In [16]:
cnxn.execute(sql)

cnxn.commit()

### 03.g. Populate room type

In [17]:
sql = '''
IF OBJECT_ID('room_type') IS NOT NULL
    DROP TABLE room_type;

CREATE TABLE room_type (
    room_type_id int identity(1,1) ,
    room_type varchar(255) NOT NULL,
    PRIMARY KEY (room_type_id),
	CONSTRAINT UC_roomtype UNIQUE (room_type )
);

INSERT INTO room_type
select distinct room_type from [dbo].[airbnb_base]
'''

In [18]:
cnxn.execute(sql)

cnxn.commit()

### 03.h. Populate listing table

In [19]:
sql = '''
IF OBJECT_ID('listing') IS NOT NULL
    DROP TABLE listing;

CREATE TABLE listing (
    listing_id int,
	host_id int, 
    name varchar(255) NOT NULL,
    price money NOT NULL,
	quality_price_outlier bit NOT NULL,
    minimum_nights int NOT NULL, 
	quality_minimum_nights_long bit NOT NULL,
    availability_365 int NOT NULL,
	quality_availability_365 bit NOT NULL,
	location_id int NOT NULL,
	room_type_id int NOT NULL,
    PRIMARY KEY (listing_id),
	FOREIGN KEY (location_id)	REFERENCES location(location_id), 
	FOREIGN KEY (room_type_id)	REFERENCES room_type(room_type_id)

);

--note the same location id is reused ---
INSERT INTO listing
select distinct 
[id],[host_id],[name],[price],[quality_price_outlier], [minimum_nights], quality_minimum_nights_long, [availability_365],quality_availability_365, [location].[location_id], r.room_type_id
from [dbo].[airbnb_base]
LEFT JOIN  [dbo].[location] ON 
([airbnb_base].[latitude] = [dbo].[location].[latitude]  and  [airbnb_base].[longitude] = [dbo].[location].[longitude])
INNER JOIN room_type r ON
[airbnb_base].room_type = r.room_type



'''

In [20]:
cnxn.execute(sql)

cnxn.commit()

### 03.i. Populate review table

In [23]:
sql = '''
IF OBJECT_ID('review_summary') IS NOT NULL
    DROP TABLE review_summary;

CREATE TABLE review_summary (
    review_summary_id int IDENTITY(1,1),
    listing_id int NOT NULL,
    number_of_reviews int NOT NULL,
	last_review date  NULL,
	quality_last_review_ismissing bit NOT NULL, 
	reviews_per_month decimal(5,2),
	quality_reviews_per_month_ismissing bit NOT NULL,
	quality_num_missing int NOT NULL,
    PRIMARY KEY (review_summary_id),
	FOREIGN KEY (listing_id)	REFERENCES listing(listing_id),
);

--note the same location id is reused ---
INSERT INTO review_summary
select distinct 
id, [number_of_reviews], [last_review], quality_last_review_ismissing,  [reviews_per_month], quality_reviews_per_month_ismissing, quality_num_missing
from [dbo].[airbnb_base]

'''

In [24]:
cnxn.execute(sql)

cnxn.commit()

### 03.j. Final Query to pull out total dataset

In [25]:
sql = '''
select 
l.[listing_id] as id, 
l.[name] as name, 
l.[host_id] as host_id,  
h.[host_name_primary] as host_name_primary, 
h.[host_name_secondary] as host_name_secondary, 
n.[neighbourhood] as neighbourhood,
loc.[location_id], 
loc.[latitude], 
loc.[longitude], 
room.room_type,
l.[price], 
l.[minimum_nights], 
r.[number_of_reviews], 
r.[last_review], 
r.[reviews_per_month],
h.[calculated_host_listings_count],
l.[availability_365], 
l.[quality_minimum_nights_long],
l.quality_availability_365,
l.quality_price_outlier,
r.quality_last_review_ismissing,
r.quality_reviews_per_month_ismissing,
r.quality_num_missing

from [dbo].[listing] l
INNER JOIN host h on h.host_id = l.host_id
INNER JOIN location loc on loc.[location_id] = l.location_id
INNER JOIN neighbourhood n on n.neighbourhood_id = loc.neighbourhood_id
INNER JOIN review_summary r on r.listing_id = l.listing_id
INNER JOIN room_type room on room.room_type_id = l.room_type_id
order by l.listing_id
'''

In [26]:
sql_query = pd.read_sql_query(sql,cnxn) # here, the 'conn' is the variable that contains your database connection information from step 2

df = pd.DataFrame(sql_query)
df.to_csv ('finaldataset.csv', index = False) # place 'r' before the path name to avoid any errors in the path